In [ ]:
import igl
import math
import scipy as sp
import numpy as np
import meshplot as mp
import pandas as pd
import matplotlib.pyplot as plt
import numba
import os
import csv
import pandas as pd

root_folder = os.getcwd()

In [ ]:
mesh_file_name="equilibrium.off"
mesh_file_path="/home/BU/dredwan1/membrane_dynamics_results/Particle_Interaction_Optimized_results/inside_particle/biased/rp=0.20/phi=0.9/" + mesh_file_name

absolute_file_path = os.path.abspath(mesh_file_path)
# Read the mesh from the file
v, f1 = igl.read_triangle_mesh(absolute_file_path)
#v, f1 = igl.read_triangle_mesh(os.path.join(root_folder, "dump00990000.off"))### Read The Final Mesh Here
v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "start.off") ) ### Read The Initial Mesh Here                         

In [ ]:
#Mesh with particl
rp=0.20
rho=0.1*rp
z=np.amax(v_init[:,2])-rp-1*rho
x=0
y=0
fig = plt.figure(figsize =(14, 8))
ax = plt.axes(projection ='3d')
surf=ax.plot_trisurf(v[:,0], v[:,1], v[:,2], triangles = f1, edgecolor=[[0,0,0]], linewidth=0.9, alpha=0.0, shade=False)
u, v1 = np.mgrid[0:2 * np.pi:30j, 0:np.pi:20j]
x1 = (rp*(np.cos(u) * np.sin(v1)))
y1 = (rp*(np.sin(u) * np.sin(v1)))
z1 = (rp*(np.cos(v1)))+z
surf=ax.plot_surface(x1, y1, z1)
ax.view_init(0,0)
ax = plt.gca()
ax.set_xlim([-1, 1.5])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()
ax.set_proj_type('ortho')
ax.set_box_aspect((xmax-xmin, ymax-ymin,ymax-ymin))
ax.set_zticks([])
ax.set_xticks([])
ax.set_yticks([])

In [ ]:
p=mp.plot(v,f1,shading={"wireframe":True, "wire_color": "black", #Wireframerendering   
                               "width": 300, "height": 300},return_plot=True) ###Viewing The Mesh

In [ ]:
len(f)

In [ ]:
average_edge_length=igl.avg_edge_length(v,f)


In [ ]:
def cal_volumetot(v,f):
    Volumetot = 0
    for i in range(len(f)):
        sum=0
        p0x=v[f[i][0]][0]
        p0y=v[f[i][0]][1]
        p0z=v[f[i][0]][2]
        p1x=v[f[i][1]][0]
        p1y=v[f[i][1]][1]
        p1z=v[f[i][1]][2]
        p2x=v[f[i][2]][0]
        p2y=v[f[i][2]][1]
        p2z=v[f[i][2]][2]
        v321= p2x*p1y*p0z
        v231= p1x*p2y*p0z
        v312= p2x*p0y*p1z
        v132= p0x*p2y*p1z
        v213= p1x*p0y*p2z
        v123= p0x*p1y*p2z
        sum=(-v321+ v231+ v312-v132-v213+ v123) / 6.0
        #print(sum)
        Volumetot+=sum
    return Volumetot


In [ ]:
### To chech the initial and final volume
Volume_Final= cal_volumetot(v,f1)
Volume_new=cal_volumetot(v_init,f)
print(Volume_new)
reduced_volume=Volume_new/(4/3)*np.pi
print(reduced_volume)

In [ ]:
dbl_area = igl.doublearea(v, f1)    
Areatot = np.sum(dbl_area)/2
reduced_volume=(3/4)*(1/Areatot**(3/2))
print(reduced_volume)

In [ ]:
Kb=0.01
H0=0
def Force_Bending(v,f):
    npv = igl.per_vertex_normals(v, f)
    K = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    Hn = -minv.dot(l.dot(v))/2
    H_mean = np.linalg.norm(Hn, axis=1)
    sign_H = np.sign(np.sum(Hn*npv, axis=1))
    H_mean_signed = H_mean*sign_H
    Lap_H = minv.dot(l.dot(H_mean_signed-H0))
    kn = minv.dot(K)
    first_term = 2*(H_mean_signed-H0)*(H_mean_signed**2 + H0*H_mean_signed -kn)
    totalforce = first_term + Lap_H
    Force_bending = 2*Kb*npv*totalforce[:,None]*area_voronoi[:,None]

    Eb = 2*Kb*(((H_mean_signed-H0)**2))*area_voronoi
    total_EB = np.sum(Eb)
        
    return Force_bending,total_EB

In [ ]:
### To check the bending energy of the vesicle
force,energy=Force_Bending(v,f1)
energy

In [ ]:
# with open('logfile_1.txt','r') as f: ###Give the file name here
#     Energy1=[]
#     string='Total Energy' ### Name it as the which energy term you want 
#     for line in f:
#         if string in line:
#             Energy1.append(float(line.split(': ')[1]))
# print(Energy1)

In [ ]:
###Plotting Energy Contribution with resrpect to number of iterations
import matplotlib as mpl
mpl.rcParams['figure.dpi']=400
plt.plot(number_of_iterations,BendingEnergy,'bo-',label='Bending Energy')
plt.plot(number_of_iterations,AdhesionEnergy,'g<-',label='Adhesion Energy')
plt.plot(number_of_iterations,TotalEnergy,'rs-',label='Total Energy')
plt.plot(number_of_iterations,VolumeEnergy,'y<-',label='Volume Energy')
plt.plot(number_of_iterations,BiasedEnergy,'k.-',label='Biased Energy')
csfont={'fontname':'Times New Roman'}
plt.xlabel('Number of Iterations',**csfont)
plt.ylabel('Energy of the System',fontsize=12,**csfont)
plt.title('Inside_Particle_u=2.1,rp=0.15,K_biased=1.0')
plt.legend(loc='upper right')
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

CALCULATING WRAPPING FRACTION_


In [ ]:
Wrapping_Fraction_020=np.array([0.452346,0.67696,0.7472,0.82429,0.87866,0.8398864,0.9241084,0.86083088])
Wrapping_Fraction_030=np.array([0.503228300,0.6344194,0.738762,0.802154,0.87590,0.92736,0.9220111,0.92555443])
Wrapping_Fraction_015=np.array([0.375892,0.63448709,0.6645164,0.75456950,0.9119146,0.74677488,0.94166674,0.942738389])
Rescaled_Adhesion_Strength=np.array([1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0])
import matplotlib as mpl
mpl.rcParams['figure.dpi']=300
plt.plot(Rescaled_Adhesion_Strength,Wrapping_Fraction_015,'kx-.',label='Particle Radius=0.15')
plt.plot(Rescaled_Adhesion_Strength,Wrapping_Fraction_030,'bo--',label='Particle Radius=0.30')
plt.plot(Rescaled_Adhesion_Strength,Wrapping_Fraction_020,'g<-.',label='Particle Radius=0.20')
csfont={'fontname':'Times New Roman'}
plt.xlabel('Rescaled Adhesion Strength',**csfont)
plt.ylabel('Wrapping Fraction',fontsize=12,**csfont)
#plt.ylim((0.3, 1.0))
plt.xticks(np.arange(1.5, 6.0, step=0.5)) 
plt.yticks(np.arange(0.3, 0.9, step=0.1)) 
plt.title('Inside_Particle_u=2.0,w/o angle_condition,Ka=2.0')
plt.legend(loc='right')

Reading Information from  log file

In [ ]:
import pandas as pd
data = pd.read_csv("Insider_Particle_Energy.csv")
wrapping_fracction=np.array(data.iloc[:,0])
total_energy=np.array(data.iloc[:,1])

In [ ]:
data = pd.read_csv("Insider_Particle_Bending_Energy.csv")
wrapping_fracction_bending=np.array(data.iloc[:,0])
bending_energy=np.array(data.iloc[:,1])

In [ ]:
# directory path where the text file is located
dir_path = "/home/BU/dredwan1/membrane_dynamics_results/Particle_Interaction_Optimized_results/inside_particle/biased/rp=0.20/phi=0.9"

# name of the text file
file_name = "logfile.txt"

# join the directory path and file name to get the absolute file path
file_path = os.path.join(dir_path, file_name)

# read the text file into a pandas DataFrame
df = pd.read_csv(file_path, skiprows=1,skipfooter=1,delim_whitespace=True,engine='python')
df.columns = ['Iteraion','Time', 'Area', 'Volume', 'ReducedVolume', 'BendingEnergy', 
              'AreaEnergy', 'VolumeEnergy', 'AdhesionEnergy', 'BiasedWrappingEnergy',
              'TotalEnergy', 'EnergyChangeRate', 'ForceResidual']


In [ ]:
Bending_Energy= df.iloc[-1]['BendingEnergy']
Total_Energy=df.iloc[-1]['TotalEnergy']
Reduced_Volume=df.iloc[-1]['ReducedVolume']
Adhesion_Energy=df.iloc[-1]['AdhesionEnergy']
print(Bending_Energy)
print(Reduced_Volume)

In [ ]:
u=2
Kb=0.01
Rp=0.20
U=(u*Kb)/Rp**2
Wrapping_Fraction=Adhesion_Energy/(U*4*np.pi*(Rp**2)) 
print(Wrapping_Fraction)

In [ ]:
Wrapping_Fraction_biased=[0.2980,0.36224142,0.4237,0.48019,0.54273427,0.6154402283156275,0.692455305,0.7424180,0.7650737269] 
Bending_Energy_Biased=np.array([0.264799,0.273371,0.282844,0.292814,0.305507,0.322168,0.340929,0.354718,0.36145])/(8*np.pi*Kb)
Total_Energy_Biased=np.array([0.202284,0.181181,0.17416,0.169682,0.166918,0.169179])/(8*np.pi*Kb)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi']=300
plt.plot(wrapping_fracction_bending,bending_energy,'b--',label='Theoretical')
plt.plot(Wrapping_Fraction_biased,Bending_Energy_Biased,'rs',label='u=2.0_biased_with_volume_constraint')
# plt.plot(Wrapping_Fraction_biased_rv,Bending_Energy_Biased_rv,'<',label='u=2.0_biased_withour_volume_constraint')

#plt.plot(Rescaled_Adhesion_Strength,Wrapping_Fraction_2,'g<-.',label='Particle Radius=0.20')
csfont={'fontname':'Times New Roman'}
plt.ylabel('Bending Energy/ 8*pi*Kb',**csfont)
plt.xlabel('Wrapping Fraction',fontsize=12,**csfont)
#plt.xlim((0.5, 5.5))
plt.xticks(np.arange(0.0, 1.0, step=0.1)) 
# plt.yticks(np.arange(0.1, 0.9, step=0.1)) 
##plt.title('Inside_Particle_u=2.1,rp=0.15,K_biased=1.0')
plt.legend(loc='upper left')

In [ ]:
# reduced_volume_1280=[0.63504,0.68778,0.74051,0.79323,0.845890,0.94361]
# oblate_numerical_1280=[0.469003/(8*3.14*0.01), 0.43625/(8*3.14*0.01),0.40417/(8*3.14*0.01),0.37260/(8*3.14*0.01),
#    0.34105/(8*3.14*0.01),0.28318/(8*3.14*0.01)]
reduced_volume_5120=[0.63336,0.687693,0.742101,0.796516,0.850845,0.904886,0.958345,0.999357]
oblate_numerical_5120=[0.467539/(8*3.14*0.01),0.432937/(8*3.14*0.01),0.398996/(8*3.14*0.01),
                       0.365826/(8*3.14*0.01),0.333561/(8*3.14*0.01),
   0.302429/(8*3.14*0.01),0.272782/(8*3.14*0.01),0.251172/(8*3.14*0.01)]

X_prolate=[0.9940438871473354, 0.9463949843260187, 0.8887147335423196, 0.8360501567398118, 0.7796238244514107, 0.7244514106583071,0.6855799373040751,0.6579937304075234, 0.6291536050156739,0.6078369905956111]
Y_prolate=[1.011599266811777, 1.0977801268498946, 1.2005306241473042, 1.3066242097918122, 1.4259912100730066, 1.5652969724741979, 1.6780923567210655, 1.7876123475562127, 1.9270274633145523, 2.0199621949822433]


X_oblate=[0.9840920486789093, 0.9452574637651237,0.8943557949256427, 0.8421091441077404,0.795214105793451,
   0.7563559247046054,0.7148219373639533,0.6839958942655395,
   0.649144343701886,0.5982237979223568, 0.5499696199246102, 0.5245046277998338]
Y_oblate= [1.0360947740371285, 1.1134902282340031, 1.2261147717954919,1.3457945716999273, 1.4584545095240062, 
    1.5535175997970727, 1.6520906211103181, 1.7330903910476114, 1.828188875583268, 
    1.9549475280057105, 2.085263303818451, 2.1521761572449107]
# X1=[0.9940438871473354, 0.9463949843260187, 0.8887147335423196, 0.8360501567398118, 0.7796238244514107, 
#    0.7244514106583071,0.6855799373040751,0.6579937304075234, 0.6291536050156739,0.6078369905956111]
# Y1=[1.011599266811777, 1.0977801268498946, 1.2005306241473042, 1.3066242097918122, 1.4259912100730066, 
#    1.5652969724741979, 1.6780923567210655, 1.7876123475562127, 1.9270274633145523, 2.0199621949822433]

x1=[0.717466,0.743963,0.791581,0.842263,0.895028,0.949205,0.999083]
y1=[0.398191/(8*3.14*0.01),0.379601/(8*3.14*0.01),0.351821/(8*3.14*0.01),0.32575/(8*3.14*0.01),
0.300358/(8*3.14*0.01),0.274926/(8*3.14*0.01),0.25116/(8*3.14*0.01)]



import matplotlib as mpl
csfont={'fontname':'Times New Roman'}
mpl.rcParams['figure.dpi'] = 400
f,ax=plt.subplots(figsize=(5,3.3))

plt.axis([0.5, 1, 1, 2.4])
plt.plot(reduced_volume_5120,oblate_numerical_5120,'o',linewidth=2,markersize=6,label='Oblate')
# plt.plot(reduced_volume_1280,oblate_numerical_1280,'rs',linewidth=2)
plt.plot(X_oblate,Y_oblate,'k--',linewidth=2,label='Oblate Branch')
plt.plot(X_prolate,Y_prolate,'r--',linewidth=2,label= 'Prolate Branch')
plt.plot(x1,y1,'s',linewidth=2,markersize=6,label='Prolate',color='green')
# plt.plot(x2,y2,'bs',linewidth=2)



plt.xlabel('reduced volume ',**csfont,fontsize=20)

# plt.legend(['Nv=460','Nv=1027','Nv=1662','Nv=3264','Theory(Seifert et al.)'])

plt.ylabel('Bending Energy $(E_B)/(8 \pi K_B $ )',**csfont,fontsize=12)
plt.xlabel('Reduced Volume ',**csfont,fontsize=12)
# plt.legend(['Nv=460','Nv=1027','Nv=1662','Nv=3264','Theory(Seifert et al.)'])
#plt.legend(['Theory oblate Branch','number of faces:5120','number of faces:1280','Theory prolate branch','number of faces:5120','number of faces:1280'],loc='upper right',frameon=False,fontsize=8)
plt.legend(loc='upper right')

#plt.title('Using C++',**csfont,fontsize=16)
ax.tick_params(axis="x", direction="in")
ax.tick_params(axis="y", direction="in")

plt.title("Comparison between theoretical and numerical results")
#plt.margins(x=0)
#plt.savefig('Bending_Energy.tiff')
plt.show()

In [ ]:
Wrapping_Fraction_non_biased=Wrapping_Fraction
Total_Energy_non_biased=Total_Energy/(8*np.pi*Kb)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi']=300
plt.plot(wrapping_fracction,total_energy,'bo--',label='Theoretical')
plt.plot(Wrapping_Fraction_biased_rv,Total_Energy_Biased_rv,'X',label='Biased')
#plt.plot(0.41165823918076466,Total_Energy_non_biased,'s',label='Non_Biased')
#plt.plot(Rescaled_Adhesion_Strength,Wrapping_Fraction_2,'g<-.',label='Particle Radius=0.20')
csfont={'fontname':'Times New Roman'}
plt.ylabel('Total Energy/8*pi*Kb',**csfont)
plt.xlabel('Wrapping Fraction',fontsize=12,**csfont)
#plt.xlim((0.5, 5.5))
plt.xticks(np.arange(0.0, 1.0, step=0.1)) 
# plt.yticks(np.arange(0.1, 0.9, step=0.1)) 
##plt.title('Inside_Particle_u=2.1,rp=0.15,K_biased=1.0')
plt.legend(loc='right')